In [1]:
!pip install Unidecode

     |████████████████████████████████| 245kB 2.6MB/s 


In [0]:
import pandas as pd
import string
import nltk
import numpy as np
import re
import json
import unidecode
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import minmax_scale

In [3]:
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# Load dataset from metadata file
file=open("/content/drive/My Drive/ML project'20/YelpZip/metadata")
data=file.readlines()
file.close()
for ind in range(len(data)):
    data[ind]=data[ind].split()

In [0]:
# Load reviews from reviewcontent file
file2=open("/content/drive/My Drive/ML project'20/YelpZip/reviewContent")
text=file2.readlines()
file2.close()
for i in range(len(text)):
    text[i]=text[i].split("\t")
text=[t[3] for t in text]

In [0]:
Dataset=pd.DataFrame(data,columns=["user_id",'product_id','rating','label','date'])
Dataset["review_text"]=text

In [0]:
pd.set_option('mode.chained_assignment', None)

In [0]:
Dataset['rating']=Dataset['rating'].astype('float')
Dataset['date']=pd.to_datetime(Dataset['date'])
Dataset['day']=Dataset['date'].dt.day_name()
Dataset['year']=Dataset.date.dt.year
Dataset['label']=Dataset.label.astype(int)

In [10]:
Y={}
for y in Dataset.year:
  if y in Y:
    Y[y]+=1
  else:
    Y[y]=0
Y

{2004: 2,
 2005: 426,
 2006: 2261,
 2007: 7535,
 2008: 16781,
 2009: 31907,
 2010: 54578,
 2011: 80212,
 2012: 97961,
 2013: 131225,
 2014: 180674,
 2015: 5024}

In [0]:
x=Dataset[Dataset.year== 2012]
x=x.append(Dataset[Dataset.year==2013])
x=x.append(Dataset[Dataset.year==2014])
train_data=x.sample(n=25000)

In [12]:
test_data=Dataset[Dataset.year==2015]
print(train_data.shape)
print(test_data.shape)

(25000, 8)
(5025, 8)


### **Hand Crafted Features**

In [13]:
# X_train['upper_case_word_count']=X_train['review_text'].apply(lambda x:len([y for y in x.split() if y) )
#Train Dataset
#Review centric features
train_data['word_count']=train_data['review_text'].apply(lambda x: len(x.split()))
train_data['punctuation_count']=train_data['review_text'].apply(lambda x: len(''.join(c for c in x if c in string.punctuation)))
train_data['char_count']=train_data['review_text'].apply(lambda x: len(x))
train_data['title_count']=train_data['review_text'].apply(lambda x: len([word for word in x.split() if word.istitle()]))
print("-----------review_centric features extracted-------------/")
# user-centric features
train_data['user_id_no_of_review'] = train_data.groupby('user_id')['user_id'].transform('size')
train_data['user_id_ave_rating'] = train_data.groupby('user_id')['rating'].transform('mean')
train_data['user_id_ave_no_words'] = train_data.groupby('user_id')['word_count'].transform('mean')
train_data['user_id_max_review_a_day'] = train_data.groupby(['user_id','day'])['user_id'].transform('size')
print("-----------user_centric features extracted-------------/")
#Product centric feature
train_data['product_id_no_of_review']=train_data.groupby('product_id')['product_id'].transform('size')
train_data['product_id_ave_rating']=train_data.groupby('product_id')['rating'].transform('mean')
train_data['product_id_ave_no_of_words']=train_data.groupby('product_id')['word_count'].transform('mean')
train_data['product_id_max_review_a_day']=train_data.groupby(['product_id','day'])['user_id'].transform('size')
print("-----------product_centric features extracted-------------/\n\n")

#Test dataset
test_data['word_count']=test_data['review_text'].apply(lambda x: len(x.split()))
test_data['punctuation_count']=test_data['review_text'].apply(lambda x: len(''.join(c for c in x if c in string.punctuation)))
test_data['char_count']=test_data['review_text'].apply(lambda x: len(x))
test_data['title_count']=test_data['review_text'].apply(lambda x: len([word for word in x.split() if word.istitle()]))
print("-----------review_centric features extracted-------------/")
# user-centric features
test_data['user_id_no_of_review'] = test_data.groupby('user_id')['user_id'].transform('size')
test_data['user_id_ave_rating'] = test_data.groupby('user_id')['rating'].transform('mean')
test_data['user_id_ave_no_words'] = test_data.groupby('user_id')['word_count'].transform('mean')
test_data['user_id_max_review_a_day'] = test_data.groupby(['user_id','day'])['user_id'].transform('size')
print("-----------user_centric features extracted-------------/")
#Product centric feature
test_data['product_id_no_of_review']=test_data.groupby('product_id')['product_id'].transform('size')
test_data['product_id_ave_rating']=test_data.groupby('product_id')['rating'].transform('mean')
test_data['product_id_ave_no_of_words']=test_data.groupby('product_id')['word_count'].transform('mean')
test_data['product_id_max_review_a_day']=test_data.groupby(['product_id','day'])['user_id'].transform('size')
print("-----------product_centric features extracted-------------/")

-----------review_centric features extracted-------------/
-----------user_centric features extracted-------------/
-----------product_centric features extracted-------------/


-----------review_centric features extracted-------------/
-----------user_centric features extracted-------------/
-----------product_centric features extracted-------------/


In [15]:
train_data.head()

,user_id,product_id,rating,label,date,review_text,day,year,word_count,punctuation_count,char_count,title_count,user_id_no_of_review,user_id_ave_rating,user_id_ave_no_words,user_id_max_review_a_day,product_id_no_of_review,product_id_ave_rating,product_id_ave_no_of_words,product_id_max_review_a_day
497013,29226,3183,5.0,1,2014-05-13,"Since my last review, I have been to Upstairs ...",Tuesday,2014,72,17,430,7,1,5.0,72.0,1,11,4.000000,131.636364,1
408918,31059,874,4.0,1,2013-06-16,Just want to update that they now take reserva...,Sunday,2013,20,6,111,3,5,4.2,78.4,2,10,3.000000,93.400000,1
120624,85382,1091,2.0,-1,2012-07-11,"This place is over-rated in my opinion, the bu...",Wednesday,2012,52,11,283,5,1,2.0,52.0,1,11,3.000000,121.727273,3
525270,13879,4397,5.0,1,2013-06-18,Ethiopian food is something that I knew nothin...,Tuesday,2013,308,65,1774,69,2,3.5,339.0,1,9,4.666667,133.888889,1
159006,105102,4223,1.0,1,2014-04-19,ABSOLUTELY the most terrible customer service ...,Saturday,2014,61,8,342,6,1,1.0,61.0,1,67,3.880597,109.164179,7


### **Text Preprocessing**

In [0]:
#Function for expanding all the contractions in the paragraph
with open("/content/drive/My Drive/ML project'20/Contractions.json",'r') as file:
   Contractions=json.load(file)
c_re = re.compile('(%s)' % '|'.join(Contractions.keys()))

def expandContractions(text, c_re=c_re):
    def replace(match):
        return Contractions[match.group(0)]
    return c_re.sub(replace, text)

In [0]:
# lowercase,remove digits,punctuations,expand contractions,stopwords and lemmatization
stop_words = stopwords.words('english')
lemmatizer = WordNetLemmatizer()
train_data['review_text']=train_data['review_text'].apply(lambda x:unidecode.unidecode(x))  #Remove accented characters
train_data['review_text']=train_data['review_text'].apply(lambda x: expandContractions(x))  # expand contractions(eg. I've to I have) 
train_data['review_text']=train_data['review_text'].apply(lambda x:x.lower())   #Convert into lowercase
train_data['review_text']=train_data['review_text'].apply(lambda x : ''.join([c for c in x if not c.isdigit()])) #Remove numeric digits
train_data['review_text']=train_data['review_text'].apply(lambda x:  ''.join([c for c in x if c not in string.punctuation]))  # Remove punctuations
# train_data['review_text']=train_data['review_text'].apply(lambda x: ' '.join([word for word in word_tokenize(x) if word not in stop_words]))

# Test Dataset
test_data['review_text']=test_data['review_text'].apply(lambda x: unidecode.unidecode(x))
test_data['review_text']=test_data['review_text'].apply(lambda x: expandContractions(x))
test_data['review_text']=test_data['review_text'].apply(lambda x:x.lower())   #Convert into lowercase
test_data['review_text']=test_data['review_text'].apply(lambda x : ''.join([c for c in x if not c.isdigit()])) #Remove numeric digits
test_data['review_text']=test_data['review_text'].apply(lambda x:  ''.join([c for c in x if c not in string.punctuation]))  # Remove punctuations
# X_test['review_text']=X_test['review_text'].apply(lambda x: ' '.join([word for word in word_tokenize(x) if word not in stop_words]))

#### **Save Train and test csv file**

In [0]:
train_data.to_csv("/content/drive/My Drive/ML project'20/YelpZip/train.csv",index=False)
test_data.to_csv("/content/drive/My Drive/ML project'20/YelpZip/test.csv",index=False)